In [8]:

from keybert import KeyBERT
from sentence_transformers import SentenceTransformer
import numpy as np

# Preprocess 

## cleaning

In [9]:
import pandas as pd
import pandas as pd
import numpy as np
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
df = pd.read_csv('../data/raw/nlp/mtsamples/mtsamples.csv')
df = df.dropna()
df['transcription'] = df['transcription'].tolist()
df #= df.head(200)

def convert(lst_kw):
    '''This function converts the keywords to a list of keywords'''
    list_keywords = lst_kw.split(',')
    return list_keywords
df['keywords']=df['keywords'].fillna("")
df['keywords_list']= df['keywords'].apply(lambda x: x.split(','))



def location_indices(stringstext, check_list):
    '''this function finds the location of the keywords in the string'''
    
    res = dict()
    for ele in check_list :
        if ele in stringstext:
            # getting front index
            strt = stringstext.index(ele)
            
            # getting ending index
            res[ele] = [strt, strt + len(ele) - 1]
    return res.values()
        
df['location'] = df.apply(lambda x: location_indices(x.transcription, x.keywords_list), axis=1) 



def location_indices(stringstext, check_list):
    '''this function finds the location of the keywords in the string'''
    
    res = dict()
    for ele in check_list :
        if ele in stringstext:
            # getting front index
            strt = stringstext.index(ele)
            
            # getting ending index
            res[ele] = [strt, strt + len(ele) - 1]
    return res.values()
        
df['location'] = df.apply(lambda x: location_indices(x.transcription, x.keywords_list), axis=1) 


import string
def cleaning(sentence):
    '''This function takes a column and cleans it by removing punctuation, stopwords, and lemmatizing
    NEXT STEPS : personalize the stop words list, check for different aspects of the words
    '''
    
    # Basic cleaning
    sentence = sentence.strip() ## remove whitespaces
    sentence = sentence.lower() ## lowercase 
    sentence = ''.join(char for char in sentence if not char.isdigit()) ## remove numbers
    
    # Advanced cleaning
    for punctuation in string.punctuation:
        sentence = sentence.replace(punctuation, '') ## remove punctuation
    
    tokenized_sentence = word_tokenize(sentence) ## tokenize 
    stop_words = set(stopwords.words('english')) ## define stopwords
    
    tokenized_sentence_cleaned = [ ## remove stopwords
        w for w in tokenized_sentence if not w in stop_words
    ]

    lemmatized = [
        WordNetLemmatizer().lemmatize(word, pos = "v") 
        for word in tokenized_sentence_cleaned
    ]
    
    cleaned_sentence = ' '.join(word for word in lemmatized)
    
    return cleaned_sentence
df["transcription"] = df["transcription"].apply(cleaning)

[nltk_data] Downloading package stopwords to /Users/tara-
[nltk_data]     sophiatumbraegel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/tara-
[nltk_data]     sophiatumbraegel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/tara-
[nltk_data]     sophiatumbraegel/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/tara-
[nltk_data]     sophiatumbraegel/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


df.medical_specialty.unique()

# prepare data from the medical specialty

In [10]:
df.medical_specialty.unique()
dict_change = {' Allergy / Immunology':0, ' Bariatrics':1,
       ' Cardiovascular / Pulmonary':2, ' Dentistry':3, ' Urology':4,
       ' General Medicine':5, ' Surgery':6, ' Speech - Language':7,
       ' SOAP / Chart / Progress Notes':8, ' Sleep Medicine':9,
       ' Rheumatology':10, ' Radiology':11, ' Psychiatry / Psychology':12,
       ' Podiatry':13, ' Physical Medicine - Rehab':14,
       ' Pediatrics - Neonatal':15, ' Pain Management':16, ' Orthopedic':17,
       ' Ophthalmology':18, ' Office Notes':19, ' Obstetrics / Gynecology':20,
       ' Neurosurgery':21, ' Neurology':22, ' Nephrology':23, ' Letters':24,
       ' Lab Medicine - Pathology':25, ' IME-QME-Work Comp etc.':26,
       ' Hospice - Palliative Care':27, ' Hematology - Oncology':28,
       ' Gastroenterology':29, ' ENT - Otolaryngology':30, ' Endocrinology':31,
       ' Emergency Room Reports':32, ' Discharge Summary':33,
       ' Diets and Nutritions':34, ' Dermatology':35,
       ' Cosmetic / Plastic Surgery':36, ' Consult - History and Phy.':37,
       ' Chiropractic':38}

#dict_change = {' Allergy / Immunology':0, 
 #              ' Bariatrics':1,' Cardiovascular / Pulmonary': 2, ' Dentistry': 3, ' Urology': 4,
 #      ' General Medicine': 5, ' Surgery': 6, ' Gastroenterology': 7 }


df['labels_val'] = df['medical_specialty'].map(dict_change)
#df2=df.replace({"medical_specialty": dict_change})

df = df.dropna()
#df['medical_specialty'].map(dict_change)
df.head(2)

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords,keywords_list,location,labels_val
0,0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,subjective yearold white female present compla...,"allergy / immunology, allergic rhinitis, aller...","[allergy / immunology, allergic rhinitis, al...","([70, 79], [519, 525], [490, 502], [1036, 1044...",0
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,past medical history difficulty climb stairs d...,"bariatrics, laparoscopic gastric bypass, weigh...","[bariatrics, laparoscopic gastric bypass, we...","([1401, 1412], [1386, 1392], [658, 664], [1406...",1


# Supervised and keybert 

## load the data

In [11]:
# train a model 
from datasets import Dataset, DatasetDict
from datasets import load_dataset
import pandas as pd

#df = pd.read_csv('../data/raw/mtsamples.csv')
#df['transcription'] = df['transcription'].tolist()
df.to_csv('../data/processed/nlp/mtsamples/mtsamples_cleaned_test.csv', index=False)
#dataset = load_dataset('../data/raw/mtsamples.csv')
dataset = load_dataset('csv', data_files='../data/processed/nlp/mtsamples/mtsamples_cleaned_test.csv')
#df.transcription=df.transcription.astype(str)
dataset = dataset['train']
dataset = dataset.train_test_split(test_size=0.1)
dataset

Using custom data configuration default-8f0a0c89f09a597c


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /Users/tara-sophiatumbraegel/.cache/huggingface/datasets/csv/default-8f0a0c89f09a597c/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'description', 'medical_specialty', 'sample_name', 'transcription', 'keywords', 'keywords_list', 'location', 'labels_val'],
        num_rows: 3435
    })
    test: Dataset({
        features: ['Unnamed: 0', 'description', 'medical_specialty', 'sample_name', 'transcription', 'keywords', 'keywords_list', 'location', 'labels_val'],
        num_rows: 382
    })
})

In [12]:
dataset['train']

Dataset({
    features: ['Unnamed: 0', 'description', 'medical_specialty', 'sample_name', 'transcription', 'keywords', 'keywords_list', 'location', 'labels_val'],
    num_rows: 3435
})

## data to tokenized data

In [13]:
from transformers import AutoTokenizer, DataCollatorWithPadding
checkpoint = 'emilyalsentzer/Bio_ClinicalBERT' #"bert-base-cased"
#tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
#model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
def tokenize_function(examples):
    return tokenizer(examples["transcription"], padding=True, truncation=True, max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True)
from transformers import AutoModelForSequenceClassification
tokenized_dataset = tokenized_dataset.remove_columns(['Unnamed: 0', 'description', 'medical_specialty', 'sample_name', 'transcription', 'keywords', 'keywords_list', 'location'])
tokenized_dataset = tokenized_dataset.rename_column("labels_val", "labels")
tokenized_dataset.set_format("torch")

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

## train the model

In [14]:
#metric = load_metric("f1",  average="macro")
import wandb
from transformers import Trainer, TrainingArguments

from transformers import AutoModelForSequenceClassification

# 1. Start a W&B run
wandb.init(project='gpt3')

# 2. Save model inputs and hyperparameters
#config = wandb.config
#config.learning_rate = config.learning_rate


# Model training code here
# 3. Log metrics over time to visualize performance
#export WANDB_PROJECT=test01
#export TASK_NAME=sentence_prediction



from datasets import load_metric

metric = load_metric("accuracy", average="macro")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

#smaller dataset for testing purpose
small_train_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(10))
small_eval_dataset = tokenized_dataset["test"].shuffle(seed=42).select(range(2))

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=39)
from transformers import Trainer, TrainingArguments
training_arguments = TrainingArguments('test_trainer', evaluation_strategy="epoch",
                                       per_device_train_batch_size=1, 
                                       per_device_eval_batch_size=1, num_train_epochs=1,
                                       report_to = 'wandb'
                                       )


trainer = Trainer(model=model, args=training_arguments, 
                  train_dataset=small_train_dataset,
                 eval_dataset = small_eval_dataset,
                  #data_collator=data_collator,
                  tokenizer=tokenizer, compute_metrics=compute_metrics)
trainer.train()
trainer.save_model('../models/mtsamples')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: taratumbraegel (nlp_masterthesis). Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...



Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to ../models/mtsamples
Configuration saved in ../models/mtsamples/config.json


{'eval_loss': 4.030138969421387, 'eval_accuracy': 0.0, 'eval_runtime': 6.3244, 'eval_samples_per_second': 0.316, 'eval_steps_per_second': 0.316, 'epoch': 1.0}
{'train_runtime': 116.9805, 'train_samples_per_second': 0.085, 'train_steps_per_second': 0.085, 'train_loss': 2.898737144470215, 'epoch': 1.0}


Model weights saved in ../models/mtsamples/pytorch_model.bin
tokenizer config file saved in ../models/mtsamples/tokenizer_config.json
Special tokens file saved in ../models/mtsamples/special_tokens_map.json


wandb: Network error (ReadTimeout), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ReadTimeout), entering retry loop.
--- Logging error ---
Traceback (most recent call last):
  File "/opt/anaconda3/envs/nlp_masterthesis/lib/python3.10/logging/__init__.py", line 1104, in emit
    self.flush()
  File "/opt/anaconda3/envs/nlp_masterthesis/lib/python3.10/logging/__init__.py", line 1084, in flush
    self.stream.flush()
OSError: [Errno 28] No space left on device
Call stack:
  File "/opt/anaconda3/envs/nlp_masterthesis/lib/python3.10/threading.py", line 966, in _bootstrap
    self._bootstrap_inner()
  File "/opt/anaconda3/envs/nlp_masterthesis/lib/python3.10/threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/envs/nlp_masterthesis/lib/python3.10/site-packages/wandb/sdk/internal/inter

: 

## Call KeyBert with the model name and the number of keywords to extract

In [1]:
!pip install keyphrase_vectorizers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 5.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 6.0 MB/s eta 0:00:0000:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.22.2
    Uninstalling transformers-4.22.2:
      Successfully uninstalled transformers-4.22.2


In [22]:
# https://towardsdatascience.com/enhancing-keybert-keyword-extraction-results-with-keyphrasevectorizers-3796fa93f4db
from transformers import BertForMaskedLM, BertTokenizer
from keyphrase_vectorizers import KeyphraseCountVectorizer #- instead of n gram

from transformers.pipelines import pipeline
model = BertForMaskedLM.from_pretrained('../models/mtsamples') #/model
tokenizer = BertTokenizer.from_pretrained('../models/mtsamples')
hf_model = pipeline('feature-extraction', model=model, tokenizer=tokenizer)
example_text = "I am a sentence for which I'd like to get its embedding. i am sick my neck hurts my head hurts"
kw_model = KeyBERT(model=hf_model)
keywords = kw_model.extract_keywords(example_text, vectorizer=KeyphraseCountVectorizer(), # keyphrase_ngram_range=(1, 2), 
                                     stop_words='english', 
                                     use_maxsum=True, 
                                     nr_candidates=10, 
                                     top_n=5, 
                                     use_mmr=True, 
                                     diversity=0.7)

keywords



Saving model checkpoint to ../models/mtsamples
Configuration saved in ../models/mtsamples/config.json
Model weights saved in ../models/mtsamples/pytorch_model.bin
tokenizer config file saved in ../models/mtsamples/tokenizer_config.json
Special tokens file saved in ../models/mtsamples/special_tokens_map.json
loading configuration file ../models/mtsamples/config.json
Model config BertConfig {
  "_name_or_path": "emilyalsentzer/Bio_ClinicalBERT",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15

[('sick neck', 0.8674),
 ('sentence like', 0.8549),
 ('embedding sick', 0.7738),
 ('head', 0.6698),
 ('sentence', 0.6611)]

wandb: Network error (ConnectionError), entering retry loop.


In [19]:

from keybert import KeyBERT
model = KeyBERT(model='../models/mtsamples')
kw = model.extract_keywords('I have a headache and a fever', keyphrase_ngram_range=(1, 3), stop_words='english', 
                       use_maxsum=True, nr_candidates=20, top_n=5, use_mmr=True)

kw  # [('fever', 0.0), ('headache', 0.0)]

No sentence-transformers model found with name ../models/mtsamples. Creating a new one with MEAN pooling.
loading configuration file ../models/mtsamples/config.json
Model config BertConfig {
  "_name_or_path": "../models/mtsamples",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "LABEL_23",
    "24": "LABEL_24",
    "25": "LAB

[('headache fever', 0.8847), ('headache', 0.7987), ('fever', 0.7897)]

wandb: Network error (ConnectionError), entering retry loop.


## apply Bert to Dataframe

In [88]:

from keybert import KeyBERT
model = KeyBERT(model='../models/mtsamples')
df['keywords_outcome'] = df.apply(lambda x: model.extract_keywords(x['transcription'], 
                                                                   keyphrase_ngram_range=(1, 2), 
                                                                   stop_words='english',  
                                                                   use_maxsum=True, 
                                                                   nr_candidates=25, 
                                                                   top_n=10, 
                                                                   use_mmr=True), axis=1)
df.head()

KeyboardInterrupt: 

In [ ]:
df.to_csv('../data/processed/mtsamples_processedBert.csv', index=False)